In [1]:
import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer

import torch
from transformers import AutoTokenizer, AutoModel
import plotly.express as px
import datetime

# Генерация эмбеддингов для строк

In [2]:
print(datetime.datetime.now())

2022-03-09 01:43:53.519274


## Загрузка данных

In [3]:
df_education = pd.read_csv('data/uuid_x_education_fields.csv', index_col=0)

## LaBSE

In [9]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

sentences = ["Hello World", "Привет Мир"]
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)
    
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
print(embeddings)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-0.0714, -0.0772, -0.0305,  ...,  0.0136, -0.0402, -0.0245],
        [-0.0644, -0.0687, -0.0521,  ...,  0.0139, -0.0362, -0.0213]])


https://huggingface.co/cointegrated/LaBSE-en-ru

In [4]:
def get_education_enbeddings(df_education):
    
    tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
    model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

    embedding_vec = []
    df_education_study = df_education[['education_field_of_study', 'education_school_name']].drop_duplicates().reset_index(drop=True)

    for i, row in enumerate(df_education_study['education_field_of_study']):
        
        print('{} / {}'.format(i+1, df_education_study.shape[0]), end="\r")

        embeddings = []

        if row == row:
            encoded_input = tokenizer(row, padding=True, truncation=True, max_length=64, return_tensors='pt')

            with torch.no_grad():
                model_output = model(**encoded_input)

            embeddings = model_output.pooler_output
            embeddings = torch.nn.functional.normalize(embeddings)

        embedding_vec.extend(np.array(embeddings))

    df_embeddings = pd.DataFrame(embedding_vec)
    
    df_embeddings = pd.concat(
        [
            df_embeddings, 
            df_education_study
        ], axis=1
    )
    
    return df_embeddings

In [5]:
df_embeddings = get_education_enbeddings(df_education)
df_embeddings.to_csv('data/embeddings/education_field_of_study.csv')

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
df_embeddings = pd.read_csv('data/embeddings/education_field_of_study.csv', index_col=0)
vec_embedding = df_embeddings.loc[:, ~df_embeddings.columns.isin(
    ['education_field_of_study', 'education_school_name']
)]

In [29]:
df_education

,org_uuid,education_school_name,education_field_of_study,education_description,education_degree,education_start_date_year,education_end_date,education_school_link
0,438c7e89-ba0c-b8cc-7ea6-1a0ac47ae40d,Davidson College,English,NaN,BA,NaN,NaN,https://www.linkedin.com/school/davidson-college/
0,438c7e89-ba0c-b8cc-7ea6-1a0ac47ae40d,Queen Mary University of London,"English, History",Semester study abroad experience.,NaN,NaN,NaN,https://www.linkedin.com/school/queen-mary-uni...
0,b37187f2-8ad4-8225-cfa2-2757f2ef9bc4,Kyungpook National University,"Electrical, Electronics and Communications Eng...",NaN,Bachelor of Science (BS),1985.0,1989,https://www.linkedin.com/school/%EA%B2%BD%EB%B...
0,75e76c56-3026-404d-079d-be335a63d9ff,Highline College,NaN,"Undergraduate Studies, One Year \nTechnology S...",NaN,NaN,NaN,https://www.linkedin.com/school/highline-college/
0,75e76c56-3026-404d-079d-be335a63d9ff,Highline Community Collage,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
0,3a372b39-86d8-a24e-9fea-3be65013b476,Harpur College,Political Behavior,NaN,Bachelor of Arts - BA,1959.0,1963,NaN
0,fcc72760-5eb8-a34e-718a-5f0e02ec916f,University of Florida College of Medicine,NaN,NaN,Doctor of Medicine (M.D.),NaN,NaN,https://www.linkedin.com/school/university-of-...
0,fcc72760-5eb8-a34e-718a-5f0e02ec916f,University of Virginia,NaN,NaN,Bachelor's degree,NaN,NaN,https://www.linkedin.com/school/university-of-...
0,fcc72760-5eb8-a34e-718a-5f0e02ec916f,Yale School of Management,NaN,NaN,MBA,NaN,NaN,https://www.linkedin.com/school/yale-school-of...
